<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html lecture9.do.txt --no_mako -->
<!-- dom:TITLE: Quantum algorithms (VQE) and nuclear physics with applications, parts I and II    -->

# Quantum algorithms (VQE) and nuclear physics with applications, parts I and II   
**Benjamin Hall and Morten Hjorth-Jensen**, [Facility for Rare Isotope Beams and National Superconducting Cyclotron Laboratory](http://www.frib.msu.edu/) and [Department of Physics and Astronomy](https://www.pa.msu.edu/), [Michigan State University](http://www.msu.edu/), East Lansing, MI 48824, USA

Date: **Jun 20, 2022**

Copyright 2013-2022, Benjamin Hall and Morten Hjorth-Jensen. Released under CC Attribution-NonCommercial 4.0 license

## Overview

In these two lectures we will derive the essential elements for using
the VQE algorithm applied to a selected nuclear physics model, in our
case the so-called pairing Hamiltonian.  In order to do so, we will
need to present the so-called Jordan-Wigner transformation first. We
will apply this to the pairing Hamiltonian discussed previously. This
Hamiltonian is writen in second quantization and we will use the Pauli
matrices discussed earlier in order to rewrite the second-quantization
creation and annihilation operators in terms of Pauli matrices. 

The unitary coupled cluster approach is discussed below here.

Thereafter, in the second lecture,  we discuss how to implement the VQE algorithm. The results
obtained with the VQE algorithm will be compared to those obtained by
using FCI and the unitary coupled cluster method (as well as the
standard coupled cluster method with doubles excitations only).

## The Jordan-Wigner transformation

The Jordan-Wigner transformation is a transformation that maps the
Pauli gates discussed earlier onto fermionic creation and
annihilation operators. The creation and annihilation operators from
the second quantization formalism (can then be represented on quantum
computers, and we will be able to rewrite our second quantization
Hamiltonian in terms of
quantum gates.

## Explicit expressions

Suppose that we represent a qubit in state $\vert 0\rangle$ as
a state occupied with a fermion and $\vert 1 \rangle$ as a state with no
fermion. We then see that the operators

<!-- Equation labels as ordinary links -->
<div id="eq:sigmaplussigmaminus"></div>

$$
\begin{equation}
\label{eq:sigmaplussigmaminus} \tag{1}
    \sigma_+ = \frac{1}{2}(\sigma_x + i\sigma_y) = \begin{bmatrix}
    0  1  
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation} 
    0  0
\end{bmatrix} \notag 
\label{_auto1} \tag{2}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation} 
    \sigma_- = \frac{1}{2}(\sigma_x - i\sigma_y) = \begin{bmatrix}
    0  0  
\label{_auto2} \tag{3}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation} 
    1  0
\end{bmatrix},
\label{_auto3} \tag{4}
\end{equation}
$$

have the following effect on the qubit basis states
$$\sigma_+ \vert 1 \rangle = \vert 0\rangle \qquad \sigma_- \vert 0\rangle = \vert 1 \rangle,$$
and
$$\sigma_+ \vert 0\rangle = 0 \qquad \sigma_-\vert 1 \rangle = 0.$$

## Replacing creation and annihilation operators with Pauli matrices

The term$\sigma_+$ acts as a creation operator and $\sigma_-$ acts as
an annihilation operator. However, since fermionic states are
anti-symmetric, $a^\dagger_a a^\dagger_b \vert c \rangle = -
a^\dagger_b a^\dagger_a \vert c \rangle$, we need our quantum gate
representation of the creation/annihilation operators to preserve this
property. This can be achieved by multiplying the $\sigma_z$ matrix on
all the occupied states leading up to the one we operate on. The
complete creation and annihilation operators can then be represented
as

<!-- Equation labels as ordinary links -->
<div id="eq:LadderOpsPauli"></div>

$$
\begin{equation}
\label{eq:LadderOpsPauli} \tag{5}
    a^\dagger_n \equiv \left(\prod_{k=1}^{n-1}\sigma_z^k \right)\sigma_+^n \qquad a_n \equiv \left(\prod_{k=1}^{n-1}\sigma_z^k \right) \sigma_-^n
\end{equation}
$$

where the superscript tells us which qubit the operator acts on.

## Representations of spin states
For
convenience, we have opted for having that odd qubits are in a spin up state, while
even qubits are in spin down state. For example for the following
state

$$
\begin{align*}
    \textit{Qubit state:} & \vert 0 \: \; \ 0 \: \; \ 1 \: \; \ 1 \rangle \\
    \textit{Spin state:} & + \; - \; + \; - \; \\
    \textit{Spacial state:} & \ 1 \; \: \ 1 \ \; \: 2 \ \; \: 2 \; ,
\end{align*}
$$

the first spacial basis state is occupied with a fermion pair with opposite spin, while the second spacial state is not occupied with any fermions.
The equivalent of the reference state for our qubit-state is then

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation}
    \vert \Psi_0\rangle = \vert 0\rangle^{\otimes^n}\vert 1 \rangle^{\otimes^k},
\label{_auto4} \tag{6}
\end{equation}
$$

where $n$ is the number of particles and $n+k$ is the number of spin-orbitals.

## Jordan-Wigner transformation of Pairing Hamiltonian

We can write the pairing Hamiltonian in  second quantized representation in terms of the Pauli matrices. A detailed derivation is provided in appendix B. Here we simply state the result: For the one body part of the Hamiltonian we have the terms

<!-- Equation labels as ordinary links -->
<div id="eq:Onebodyintermsofpauli"></div>

$$
\begin{equation}
\label{eq:Onebodyintermsofpauli} \tag{7}
    \hat{H}_{0p} = \frac{1}{2}\delta(p - 1 - I[p\%2=0])(I^p + \sigma_z^p),
\end{equation}
$$

where we sum over each qubit $p$ and $\%$ is the modulo operator. We have that $I[f(x) = y] = 1$ if $f(x) = y$ and zero otherwise. For the interaction term we have two possibilities. First for $p = q$ we have:

<!-- Equation labels as ordinary links -->
<div id="eq:twobodypaulipisq"></div>

$$
\label{eq:twobodypaulipisq} \tag{8}
    \hat{V}_{p} = -\frac{1}{8}g\Big[ I^{\otimes^{2p-2}} \otimes I  \otimes I \otimes I^{\otimes^{n - 2p}} \nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation} 
    + I^{\otimes^{2p-2}} \otimes I  \otimes \sigma_z \otimes I^{\otimes^{n - 2p}} 
\label{_auto5} \tag{9}
\end{equation}
$$

$$
+ I^{\otimes^{2p-2}} \otimes \sigma_z  \otimes I \otimes I^{\otimes^{n - 2p}} \nonumber
$$

$$
+ I^{\otimes^{2p-2}} \otimes \sigma_z  \otimes \sigma_z \otimes I^{\otimes^{n - 2p}}\Big].\nonumber
$$

## And the remaining contribution

For $q - p \geq 1$ we get:

<!-- Equation labels as ordinary links -->
<div id="eq:twobodypauliqgeqp"></div>

$$
\label{eq:twobodypauliqgeqp} \tag{10}
       \hat{V}_{pq} = -\frac{1}{16}g [ I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q}} \nonumber
$$

$$
-I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q}} \nonumber
$$

$$
+ I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q}} \nonumber
$$

<!-- Equation labels as ordinary links -->
<div id="_auto6"></div>

$$
\begin{equation} 
         + I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q}}  
\label{_auto6} \tag{11}
\end{equation}
$$

$$
+ I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q}} \nonumber
$$

$$
+ I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q}} \nonumber
$$

$$
- I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q}} \nonumber
$$

$$
+ I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q}}].\nonumber
$$

We have included a factor of two so the sum over $p$ and $q$ here can be restricted to $q > p$.
The complete Jordan-Wigner transformed pairing Hamiltonian can then be written as

<!-- Equation labels as ordinary links -->
<div id="eq:JordanWignerHamiltonian"></div>

$$
\begin{equation}
\label{eq:JordanWignerHamiltonian} \tag{12}
    \hat{H} = \sum_p \hat{H}_{0p} + \sum_{p} \hat{V}_p + \sum_{q > p} \hat{V}_{pq},
\end{equation}
$$

where $\hat{H}_{0p}$, $\hat{V}_p$ and $\hat{V}_{pq}$ is given by the one-body part and two-body part, respectively, of the Hamiltonian.

## Details behind the derivation of the Jordan-Wigner transformed Pairing Hamiltonian

Here we show how we can rewrite our pairing Hamiltonian in terms of the Pauli matrices. First, we observe that
$$a^\dagger_i a_i = \left(\prod_{k=1}^{i-1}\sigma_z^k \right)\sigma_+^i \left(\prod_{k=1}^{i-1}\sigma_z^k \right) \sigma_-^i $$
$$ =\sigma_+^i \sigma_-^i = \frac{1}{2}(I^i + \sigma_z^i) ,$$
and see that the one-body part of the Hamiltonian can be written as

$$
\hat{H}_0 = \frac{1}{2}\xi \sum_{p} ([(p-1) - (p-1)\%2]/2)(I^p + \sigma_z^p).
$$

where $p$ now runs over each qubit and $\%$ is the modulo operator. For the interaction term, we get

<!-- Equation labels as ordinary links -->
<div id="eq:Twobodyintermsofpauli"></div>

$$
\begin{equation}
\label{eq:Twobodyintermsofpauli} \tag{13}
    \hat{V} = \notag 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto7"></div>

$$
\begin{equation} 
    -\frac{1}{2}g \sum_{pq}\left( \prod_{k=1}^{2p-2} \sigma_z^k \right)\sigma_{+}^{2p-1} \left( \prod_{k=1}^{2p-1} \sigma_z^k \right)\sigma_{+}^{2p}
    \left( \prod_{k=1}^{2q-1} \sigma_z^k \right)\sigma_{-}^{2q} \left( \prod_{k=1}^{2q-2} \sigma_z^k \right)\sigma_{-}^{2q-1}. 
\label{_auto7} \tag{14}
\end{equation}
$$

## Rewriting the Pairing Hamiltonian

We have that

$$
\begin{align*}
    \left( \prod_{k=1}^{2p-2} \sigma_z^k \right)\sigma_{+}^{2p-1} \left( \prod_{k=1}^{2p-1} \sigma_z^k \right)\sigma_{+}^{2p} = \notag \\
    \sigma_z^{\otimes^{2p-2}} \otimes \sigma_+ \otimes I^{\otimes^{n - 2p-1}} \times 
    \sigma_z^{\otimes^{2p-1}} \otimes \sigma_+ \otimes I^{\otimes^{n - 2p}} = \notag \\
    I^{\otimes^{2p-2}} \otimes \sigma_+ \sigma_z \otimes \sigma_+ \otimes I^{\otimes^{n - 2p}},
\end{align*}
$$

which can be rewritten as

$$
\begin{align*}
    \hat{V} &= \notag \\
    &-\frac{1}{2} g\sum_{pq}\left[I^{\otimes^{2p-2}} \otimes \sigma_+ \otimes \sigma_+ \otimes I^{\otimes^{n - 2p}} \right] \left[ I^{\otimes^{2q-2}} \otimes \sigma_- \otimes \sigma_- \otimes I^{\otimes^{n - 2q}} \right].
\end{align*}
$$

## Two possibilities

We have two possibilities here. First for $p=q$:

<!-- Equation labels as ordinary links -->
<div id="eq:PisQ"></div>

$$
\begin{equation}
\label{eq:PisQ} \tag{15}
    -\frac{1}{2}g \left[ I^{\otimes^{2p-2}} \otimes \sigma_+ \sigma_-  \otimes \sigma_+ \sigma_- \otimes I^{\otimes^{n - 2p}} \right],
\end{equation}
$$

and for $q - p \geq 1$ we have

<!-- Equation labels as ordinary links -->
<div id="eq:PgeqQ"></div>

$$
\begin{equation}
\label{eq:PgeqQ} \tag{16}
    -\frac{1}{2}g \left[ I^{\otimes^{2p-2}} \otimes \sigma_+ \otimes \sigma_+ \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_- \otimes \sigma_- \otimes I^{\otimes^{n-2q}} \right].
\end{equation}
$$

For $p - q \geq 1$ we simply exchange $p$ with $q$ and $\sigma_+$ with $\sigma_-$.

## More details

To continue, we insert the expressions for $\sigma_{\pm}$ into these equations. For $p = q$ we then have:

$$
\begin{align*}
   V_{p=q} = -\frac{1}{8}g\Big[ I^{\otimes^{2p-2}} \otimes & (I + \sigma_z)  \otimes (I + \sigma_z) \otimes I^{\otimes^{n - 2p}} \Big] \\
    = -\frac{1}{8}g\Big[& I^{\otimes^{2p-2}} \otimes I  \otimes I \otimes I^{\otimes^{n - 2p}} \nonumber \\
    +& I^{\otimes^{2p-2}} \otimes I  \otimes \sigma_z \otimes I^{\otimes^{n - 2p}} \nonumber \\
    +& I^{\otimes^{2p-2}} \otimes \sigma_z  \otimes I \otimes I^{\otimes^{n - 2p}} \nonumber \\
    +& I^{\otimes^{2p-2}} \otimes \sigma_z  \otimes \sigma_z \otimes I^{\otimes^{n - 2p}}\Big],\nonumber 
\end{align*}
$$

and for $q - p \geq 1$ we get

$$
-\frac{1}{32}g \left[ I^{\otimes^{2p-2}} \otimes (\sigma_x + i\sigma_y) \otimes (\sigma_x + i\sigma_y) \otimes I^{\otimes^{2(q - p - 1)}} \otimes (\sigma_x - i\sigma_y) \otimes (\sigma_x - i\sigma_y) \otimes I^{\otimes^{n-2q}} \right].
$$

## Rewriting as four-qubit operations

This can be rewritten as sixteen four-qubit operations

$$
\begin{align*}
        -\frac{1}{32}g & [ I^{\otimes^{2p_q -2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & \mp i I^{\otimes^{2p_q - 2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & \mp iI^{\otimes^{2p_q - 2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & -I^{\otimes^{2p_q -2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & \pm i I^{\otimes^{2p_q -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & + I^{\otimes^{2p_q -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & + I^{\otimes^{2p_q -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & \mp i I^{\otimes^{2p_q -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \\
        & \pm i I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        &+ I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & + I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & \mp i I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        & - I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        &\pm i I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        &\pm i I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q_p}} \nonumber \\
        &+ I^{\otimes^{2p_q -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q_p - p_q - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q_p}}],\nonumber 
\end{align*}
$$

where the subscript is used if $p > q$.

## Final expression
We can easily see that when
running over the sum over $p$ and $q$, the terms with $\pm$ and $\mp$
sign in front will cancel out. Also, we can include a factor 2 and
limit the sum to $p < q$. Therefore:

$$
\begin{align*}
      V_{p\neq q} =  -\frac{1}{16}g [& I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q}} \nonumber \\
        & -I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q}} \nonumber \\
        & + I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q}} \nonumber \\
        & + I^{\otimes^{2p -2}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q}}  \\
        &+ I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_y \otimes I^{\otimes^{n-2q}} \nonumber \\
        & + I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_x \otimes I^{\otimes^{n-2q}} \nonumber \\
        & - I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_x \otimes \sigma_x \otimes I^{\otimes^{n-2q}} \nonumber \\
        &+ I^{\otimes^{2p -2}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{2(q - p - 1)}} \otimes \sigma_y \otimes \sigma_y \otimes I^{\otimes^{n-2q}}].\nonumber 
\end{align*}
$$

## Variational Quantum Eigensolvers

As discussed in the lecture on the VQE algorithm, the variational principle states that the expectation value of the
Hamiltonian has to be larger than or equal to the ground state energy
of the system. Mathematically this can be expressed as

<!-- Equation labels as ordinary links -->
<div id="_auto8"></div>

$$
\begin{equation}
    \langle\psi\vert  H \vert \psi \rangle  \geq E_0.
\label{_auto8} \tag{17}
\end{equation}
$$

We can understand this principle intuitively by considering that no
single measurement of the energy can be lower than the ground state
energy. Hence, the expectation value of the energy can
neither. Variational methods make use of this principle by calculating
the expectation value in the baove equation for what we call a trial
wavefunction $\vert \psi_T(\boldsymbol{\theta)\rangle}$ given by

$$
\langle \psi_T(\boldsymbol{\theta})\vert H \vert\psi_T(\boldsymbol{\theta})\rangle = E(\boldsymbol{\theta}).
$$

## Variational parameters

The variational parameters $\boldsymbol{\theta} = [\theta_1, \theta_2,
\cdots, \theta_p]$ are then varied to minimize
$E(\boldsymbol{\theta})$, which hopefully makes a good approximation
for $E_0$. For variational quantum eigensolvers (VQE), the trial wave
function is given by a parametrized n-qubit state $$
U(\boldsymbol{\theta}) \vert\psi_0\rangle =
\vert\psi_T(\boldsymbol{\theta})\rangle, $$ where
$U(\boldsymbol{\theta})$ is some parametrized multi-qubit gate and
$\vert\psi_0\rangle$ is the initial state of the qubits.  As long as
the Hamiltonian can be rewritten as a sum of quantum gates $O_i$ $$ H
= \sum_{i=1}^m h_i O_i, $$ we can find its expectation value by
considering the expectation of each term

<!-- Equation labels as ordinary links -->
<div id="_auto9"></div>

$$
\begin{equation}
    \langle \psi_T(\boldsymbol{\theta})\vert  H \vert \psi_T(\boldsymbol{\theta})\rangle = \sum_{i=1}^m h_i \langle \psi_T(\boldsymbol{\theta})\vert  O_i \vert \psi_T(\boldsymbol{\theta})\rangle.
\label{_auto9} \tag{18}
\end{equation}
$$

The implementation details will be discussed in the next lecture.

## Unitary Coupled Cluster ansatz

The coupled cluster ansatz is given by 
$$\vert \psi_{CC}\rangle  = e^{\hat{T}} \vert c \rangle ,$$
with the cluster operator $\hat{T}$ and $\vert c \rangle$ being our reference state. This type of ansatz is not implementable on a quantum computer since $e^{\hat{T}}$ is not a unitary operator. Unitary coupled cluster instead suggest that we write our ansatz as

<!-- Equation labels as ordinary links -->
<div id="eq:UnitaryCoupledClusterAnsatz"></div>

$$
\begin{equation}
\label{eq:UnitaryCoupledClusterAnsatz} \tag{19}
    \vert\psi_{UCC}\rangle = e^{\hat{T} - \hat{T}^\dagger}\vert \psi_0\rangle ,
\end{equation}
$$

where $\hat{T}$ is the usual coupled cluster operator and the state $\vert \psi_0\rangle $ is the reference state.
One can show that $e^{\hat{T} - \hat{T}^\dagger}$ is unitary \[[UCCDArticle]](#UCCDArticle).

## New cluster operator

The unitary coupled cluster doubles (UCCD) method introduces the cluster operator

<!-- Equation labels as ordinary links -->
<div id="eq:UnitaryOp"></div>

$$
\begin{equation}
\label{eq:UnitaryOp} \tag{20}
    \hat{T}_{UCCD} = \hat{T} - \hat{T}^\dagger =  \notag 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto10"></div>

$$
\begin{equation} 
     \sum_{ijab} t_{ij}^{ab}(a^\dagger_a a^\dagger_b a_j a_i - a^\dagger_i a^\dagger_j a_b a_a),
\label{_auto10} \tag{21}
\end{equation}
$$

where we vary the trial wavefunction over the real cluster amplitudes $t_{ij}^{ab}$. Note that this ansatz
depends on the number of particles in our system as we sum over $i$ and $j$. Hence, we can specify the specific configuration we wish to solve for, unlike the previously discussed ansatzes. 
Rewriting eq. ([20](#eq:UnitaryOp)) in terms of Pauli gates by utilizing the Jordan-Wigner transformation results in

<!-- Equation labels as ordinary links -->
<div id="eq:UCCDoublesGates"></div>

$$
\begin{equation}
\label{eq:UCCDoublesGates} \tag{22}
    t_{ij}^{ab}(a^\dagger_a a^\dagger_b a_j a_i - a^\dagger_i a^\dagger_j a_b a_a) = \frac{it_{ij}^{ab}}{8} \bigotimes_{k=i+1}^{j-1}\sigma_z^k \bigotimes_{l=a+1}^{b-1}\sigma_z^l \notag 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto11"></div>

$$
\begin{equation} 
    (\sigma_x^i \sigma_x^j \sigma_y^a \sigma_x^b + \sigma_y^i \sigma_x^j \sigma_y^a \sigma_y^b \notag 
\label{_auto11} \tag{23}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto12"></div>

$$
\begin{equation} 
    +\sigma_x^i\sigma_y^j \sigma_y^a \sigma_y^b + \sigma_x^i \sigma_x^j \sigma_x^a \sigma_y^b \notag 
\label{_auto12} \tag{24}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto13"></div>

$$
\begin{equation} 
    - \sigma_y^i \sigma_x^j \sigma_x^a \sigma_x^b - \sigma_x^i \sigma_y^j \sigma_x^a \sigma_x^b \notag 
\label{_auto13} \tag{25}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto14"></div>

$$
\begin{equation} 
    -\sigma_y^i \sigma_y^j \sigma_y^a \sigma_x^b - \sigma_y^i \sigma_y^j \sigma_x^a \sigma_y^b ),
\label{_auto14} \tag{26}
\end{equation}
$$

where we can assume that $i < j < a < b$. The subscript denotes which qubit we act upon with the Pauli gates.

## Preparing for quantum computing, Suzuki-Trotter approximation

The preparation of this trial wavefunction is by first utilizing the Suzuki-Trotter approximation
on the operator. Denoting 
$$\hat{Z}_{ij}^{ab} =i\frac{t_{ij}^{ab}}{8p}(\bigotimes_{k=i+1}^{j-1}\sigma_z^k)(\bigotimes_{l=a+1}^{b-1}\sigma_l^z),$$ 
the Suzuki-Trotter approximation gives

<!-- Equation labels as ordinary links -->
<div id="eq:UCCTrotterApprox"></div>

$$
\begin{equation}
\label{eq:UCCTrotterApprox} \tag{27}
    \vert \psi(\boldsymbol{t})\rangle \approx  \bigg( \prod_{ijab} e^{\hat{Z}_{ij}^{ab}\sigma_x^i \sigma_x^j \sigma_y^a \sigma_x^b }
    e^{\hat{Z}_{ij}^{ab}\sigma_y^i \sigma_x^j \sigma_y^a \sigma_y^b }
    e^{\hat{Z}_{ij}^{ab}\sigma_x^i\sigma_y^j \sigma_y^a \sigma_y^b }
    e^{\hat{Z}_{ij}^{ab}\sigma_x^i \sigma_x^j \sigma_x^a \sigma_y^b } \notag 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto15"></div>

$$
\begin{equation} 
    e^{-\hat{Z}_{ij}^{ab}\sigma_y^i \sigma_x^j \sigma_x^a \sigma_x^b }
    e^{-\hat{Z}_{ij}^{ab}\sigma_x^i \sigma_y^j \sigma_x^a \sigma_x^b }
    e^{-\hat{Z}_{ij}^{ab}\sigma_y^i \sigma_y^j \sigma_y^a \sigma_x^b }
    e^{-\hat{Z}_{ij}^{ab}\sigma_y^i \sigma_y^j \sigma_x^a \sigma_y^b }
    \bigg)^p \vert c \rangle,
\label{_auto15} \tag{28}
\end{equation}
$$

where $p$ decides the step size in the Suzuki-Trotter approximation
and will be restricted to $p=1$ in our discussions.

## More details

Since
we are dealing with the pairing Hamiltonian as an example system in this school, we can
reduce the number of terms in our ansatz by not allowing to break
particle pairs. The Taylor expansion of the UCCD operator gives us

<!-- Equation labels as ordinary links -->
<div id="_auto16"></div>

$$
\begin{equation}
    e^{\sum_{ijab} t_{ij}^{ab}(a^\dagger_a a^\dagger_b a_j a_i - a^\dagger_i a^\dagger_j a_b a_a)} = I + \sum_{ijab} t_{ij}^{ab}(a^\dagger_a a^\dagger_b a_j a_i - a^\dagger_i a^\dagger_j a_b a_a) \notag 
\label{_auto16} \tag{29}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto17"></div>

$$
\begin{equation} 
     + [\sum_{ijab} t_{ij}^{ab}(a^\dagger_a a^\dagger_b a_j a_i - a^\dagger_i a^\dagger_j a_b a_a)]^2/2! \notag 
\label{_auto17} \tag{30}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto18"></div>

$$
\begin{equation} 
    + \cdots.
\label{_auto18} \tag{31}
\end{equation}
$$

We can immediately see that the first sum over $i,j,a,b$ will break
pairs if we do not introduce a restriction on the sum. This can be
explained by the fact that any non zero $t_{ij}^{ab}$ will include the
following term $$ t_{ij}^{ab}(a^\dagger_a a^\dagger_b a_j a_i -
a^\dagger_i a^\dagger_j a_b a_a)\vert c \rangle, $$ which will break
pairs if $j \neq i+1$ and $b \neq a+1$.  Hence we make the restriction
$j = i+1$ and $b = a+1$.